# CA3 - Data Driven Model
- Student Name: **Vinicius Moura Barros**
- Student Number: **T00244396**
## **Allocation of marks**

Stages 1,2,3   Total 30%

Ideally your dataset will not need a lot of work. Marks going for the choosing,  training, evaluation and refining of model primarily


---



Stage 4,5       Total 60%

Choose your model

https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html


Train and evaluate your model. Refine the model or choose another estimator or approach?

Do not overwrite cells and add commentry between iterations.



---




Stage 6         Total 10%

## **Stage 0**
### Preparation of Environment
This section outlines what needs to be installed on the Python environment before this notebook can be executed.
Python3 is being used for this project. The following packages need to be installed (e.g.: via `pip install`):
- scikit-learn
- pandas


 ### Importing the libraries

In [93]:
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
from sklearn import tree

# **Stage 1**

### Background
Overwatch is a team-based multiplayer first-person shooter developed and published by Blizzard Entertainment. Overwatch League is the official professional and competitive Overwatch league, where teams compete against each other through the year.

A regular official match is composed of a multiple games (or maps), whereas the first team to have 3 games (maps) won, wins the match. Each game is played in a different map, and each map has its own characteristics, which can be advantageous or disadvantageous to a team, depending on their playstyle.

The team who loses a map has the chance of choosing what next map they want to play. And this is exactly the type of question that this notebook will try to answer: given a team and this team's adversary, what map should they choose next, in order to increase their chances of winning the match?

### Describe Data

The dataset chosen for this notebook was a list of previous matches from Overwatch League.
All Overwatch League datasets can be found in [Overwatch League official statslab](https://overwatchleague.com/en-us/statslab), and the one used for this notebook can be downloaded exactly from [this URL](https://assets.blz-contentstack.com/v3/assets/blt321317473c90505c/blta8c8b99af29af2be/63f51f804f98c853eaf0983b/match_map_stats.csv.zip).

The dataset contains almost 14.000 rows with data about official matches that happened from 11/2018 until 05/2022.


### Loading Dataset
As the dataset may not be available during the execution of this notebook, a copy can be found in ./data/overwatch_match_map_stats.csv. By loading it from a local disk, it will also make the loading process faster.

In [94]:
DATASET = 'data/overwatch_match_map_stats.csv'
df = pd.read_csv(DATASET)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13901 entries, 0 to 13900
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   round_start_time              13901 non-null  object 
 1   round_end_time                13901 non-null  object 
 2   stage                         13901 non-null  object 
 3   match_id                      13901 non-null  int64  
 4   game_number                   13901 non-null  int64  
 5   match_winner                  13901 non-null  object 
 6   map_winner                    13901 non-null  object 
 7   map_loser                     13901 non-null  object 
 8   map_name                      13901 non-null  object 
 9   map_round                     13901 non-null  int64  
 10  winning_team_final_map_score  13901 non-null  int64  
 11  losing_team_final_map_score   13901 non-null  int64  
 12  control_round_name            4270 non-null   object 
 13  A

In [95]:
df.head()

,round_start_time,round_end_time,stage,match_id,game_number,match_winner,map_winner,map_loser,map_name,map_round,...,team_one_name,team_two_name,attacker_payload_distance,defender_payload_distance,attacker_time_banked,defender_time_banked,attacker_control_perecent,defender_control_perecent,attacker_round_end_score,defender_round_end_score
0,01/11/18 00:12,01/11/18 00:20,2018: Stage 1,10223,1,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Dorado,1,...,Los Angeles Valiant,San Francisco Shock,75.61505,0.00000,0.000000,240.000,NaN,NaN,2,0
1,01/11/18 00:22,01/11/18 00:27,2018: Stage 1,10223,1,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Dorado,2,...,Los Angeles Valiant,San Francisco Shock,75.64960,75.61505,125.750570,0.000,NaN,NaN,3,2
2,01/11/18 00:34,01/11/18 00:38,2018: Stage 1,10223,2,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Temple of Anubis,1,...,Los Angeles Valiant,San Francisco Shock,0.00000,0.00000,250.492000,240.000,NaN,NaN,2,0
3,01/11/18 00:40,01/11/18 00:44,2018: Stage 1,10223,2,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Temple of Anubis,2,...,Los Angeles Valiant,San Francisco Shock,0.00000,0.00000,225.789030,250.492,NaN,NaN,2,2
4,01/11/18 00:46,01/11/18 00:49,2018: Stage 1,10223,2,Los Angeles Valiant,Los Angeles Valiant,San Francisco Shock,Temple of Anubis,3,...,Los Angeles Valiant,San Francisco Shock,0.00000,0.00000,36.396057,250.492,NaN,NaN,4,2


# **Stage 2**

### Justification for the property that you would like the model to predict.

As described in the Background section, during a game, the team that lost a match have the chance of choosing the next map to be played. This is an important decision as by analyzing historical data, the team might take advantage of existing data about previous match to maximize their chances of winning the next match (and eventually the game).

And this is exactly the type of question that this notebook will try to answer: given a team and this team’s adversary, what map should they choose next, in order to increase their chances of winning the match?

Based on team_one_name, team_two_name, map_name and map_winner, we will try to predict what map should the team choose next.



# **Stage 3**

### Cleaning the data, linking datasets etc..



As the main objective is to predict what map to chose, we don't need all the data present in the dataset, so we will filter it to have only the data necessary to train the model.


In [96]:
df = df[['team_one_name', 'team_two_name', 'map_name', 'map_winner', 'map_loser', 'winning_team_final_map_score',
         'losing_team_final_map_score']]

In [97]:
df.head()

,team_one_name,team_two_name,map_name,map_winner,map_loser,winning_team_final_map_score,losing_team_final_map_score
0,Los Angeles Valiant,San Francisco Shock,Dorado,Los Angeles Valiant,San Francisco Shock,3,3
1,Los Angeles Valiant,San Francisco Shock,Dorado,Los Angeles Valiant,San Francisco Shock,3,3
2,Los Angeles Valiant,San Francisco Shock,Temple of Anubis,Los Angeles Valiant,San Francisco Shock,4,4
3,Los Angeles Valiant,San Francisco Shock,Temple of Anubis,Los Angeles Valiant,San Francisco Shock,4,4
4,Los Angeles Valiant,San Francisco Shock,Temple of Anubis,Los Angeles Valiant,San Francisco Shock,4,4


Also, as the data present in the dataset is mainly strings for team names and map names, we need to convert it to numerical data, so the model can understand it. For this, we will use the LabelEncoder from scikit-learn.

The helper functions below will help us to convert the data to numerical data and also to convert it back to the original data whenever needed.

In [98]:
default_dict = defaultdict(LabelEncoder)

# we need to convert the categorical data into numerical data so the model can understand it
def categorize_df(df: pd.DataFrame):
    return df.apply(lambda x: default_dict[x.name].fit_transform(x))

def prettify_df(categorized_df: pd.DataFrame):
    return categorized_df.apply(lambda x: default_dict[x.name].inverse_transform(x))

def get_map_winner_name(map_winner: int):
    res = default_dict['map_winner'].inverse_transform([map_winner])
    return res.item(0)



# **Stage 4**
Choose and train Estimator.

To help finding an estimator, the [Scikit-Learn Choosing the Right Estimator](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) was used as a guide.

The initial estimator chosen for this notebook is the Decision Tree Classifier, however if want to experiment with other classification methods like LogisticRegression and see if one is more accurate than other.

In [99]:
# These are the features to be taken into consideration when training the model
features = ['team_one_name', 'team_two_name', 'map_name']

# And target is what we want to be able to predict
target = 'map_winner'

random_state = 42
# percentage of the dataset to be used for testing (the remainder will be used for training)
test_size = 0.5

categorized_df = categorize_df(df)

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(categorized_df[features], categorized_df[target], test_size=test_size, random_state=random_state)

print(len(X_train), len(X_test), len(y_train), len(y_test))



6950 6951 6950 6951


#### DecisionTreeClassifier
Let's create an estimator with the Decision Tree Classifier

In [100]:
# create a decision tree classifier
estimator = DecisionTreeClassifier()

# fit the model on the training data
estimator.fit(X_train, y_train)

# make predictions on the testing data
y_pred = estimator.predict(X_test)

# evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7230614300100705


By using the DecisionTreeClassifier, we got an accuracy of more than 72% for the test (not used in the training) data, which is already impressive.

# **Stage 5**
### Evaluate your model and revisit 4...repeat..
Now that we have a base-line for comparison (72% of accuracy), let's experiment other estimators and see if we can get a better accuracy with other known classification models

In [101]:
estimators_to_be_tested = {
    'LogisticRegression': LogisticRegression(max_iter=10000),
    'RandomForestClassifier': RandomForestClassifier(),
    'KNeighborsClassifier':  KNeighborsClassifier(),
    'SGDClassifier': SGDClassifier(),
    'SVC': SVC(max_iter=10000),
}

for name, estimator_instance in estimators_to_be_tested.items():
    estimator_instance.fit(X_train, y_train)
    temp_y_pred = estimator_instance.predict(X_test)
    accuracy = accuracy_score(y_test, temp_y_pred)
    print(f"Accuracy for {name}: {accuracy}")

Accuracy for LogisticRegression: 0.12041432887354338
Accuracy for RandomForestClassifier: 0.7170191339375629
Accuracy for KNeighborsClassifier: 0.5110056107034959
Accuracy for SGDClassifier: 0.10991224284275644
Accuracy for SVC: 0.29247590274780605


As we can observe on the results above, given the exact same training/test data, the Decision Tree has still better results than the other ones. So, for now, we will keep using the Decision Tree Classifier.

In [102]:
import random
random_to_check = [random.randint(0,len(df)) for _ in range(10)]

to_test = categorized_df.loc[random_to_check]

# print(to_test[['team_one_name', 'team_two_name', 'map_winner']])
# test_df = pd.DataFrame(df[features])
# test_df = categorize_df(test_df)

to_test['prediction'] = estimator.predict(to_test[features])

print(to_test[['team_one_name', 'team_two_name', 'map_winner', 'prediction']])
# for map_winner_id in prediction:
#     print(map_winner_id)
#     print(f"Winner is {get_map_winner_name(map_winner_id)}")




       team_one_name  team_two_name  map_winner  prediction
12748             18              9          18          18
5577              12             13          13          13
688               13              1          13          13
12665             15              2          15          15
6978               7              3           3           3
7246               0             14          14          14
3376              19              2          19          19
12638             15              6          15          15
4438               8              1           8           8
9288              13              2           2          13


In [103]:
df.loc[9610]

team_one_name                        Paris Eternal
team_two_name                   Washington Justice
map_name                                   Numbani
map_winner                                    draw
map_loser                                     draw
winning_team_final_map_score                     3
losing_team_final_map_score                      3
Name: 9610, dtype: object

## Tooling for teams
At this stage, we'll provide a tooling for teams to be able to use the model to predict what map to choose next.

In [104]:
print("List of Teams Available")
teams = df['team_one_name'].unique()
for i, name in enumerate(teams):
    print(f"{i} - {name}")

List of Teams Available
0 - Los Angeles Valiant
1 - Los Angeles Gladiators
2 - Shanghai Dragons
3 - Seoul Dynasty
4 - Dallas Fuel
5 - Florida Mayhem
6 - London Spitfire
7 - Houston Outlaws
8 - Philadelphia Fusion
9 - New York Excelsior
10 - Boston Uprising
11 - San Francisco Shock
12 - Hangzhou Spark
13 - Toronto Defiant
14 - Atlanta Reign
15 - Chengdu Hunters
16 - Guangzhou Charge
17 - Paris Eternal
18 - Washington Justice
19 - Vancouver Titans
20 - Vegas Eternal
21 - Seoul Infernal


In [129]:
my_team, opponent_team = None, None

while my_team is None:
    team = int(input(f"What is your team? (0-{len(teams)})"))
    if 0 <= team < len(teams):
        print(f"Your team is {teams[team]}")
        my_team = team
    else:
        print("Invalid choice, please try again")

while opponent_team is None:
    team = int(input(f"What is your opponent team?(0-{len(teams)}) It must not be {teams[my_team]} "))
    if 0 <= team < len(teams):
        print(f"Your opponent team is {teams[team]}")
        opponent_team = team
    else:
        print("Invalid choice, please try again")

Your team is Boston Uprising
Your opponent team is Chengdu Hunters


In [133]:
unique_maps = list(df['map_name'].unique())


dict_data = {
             'map_name': unique_maps,
              'team_one_name': [teams[my_team]] * len(unique_maps),
              'team_two_name': [teams[opponent_team]] * len(unique_maps)
             }
predict_df = pd.DataFrame(dict_data)

categorized_predict_df = categorize_df(predict_df)
print(categorized_predict_df)
estimator.predict(categorized_predict_df[features])

    map_name  team_one_name  team_two_name
0          4              0              0
1         24              0              0
2         11              0              0
3         18              0              0
4          5              0              0
5         12              0              0
6         19              0              0
7         10              0              0
8         14              0              0
9         25              0              0
10        16              0              0
11        13              0              0
12        23              0              0
13         9              0              0
14         7              0              0
15        26              0              0
16         0              0              0
17        22              0              0
18         1              0              0
19        21              0              0
20         8              0              0
21        15              0              0
22         

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0])

# **Stage 6**
***Present results and Conclusions***

